In [2]:
import pandas as pd

df_titanic = pd.read_csv('../data/raw/train.csv') # Al cargar el dataset y convertirlo en un DataFrame Los valores faltantes se representan como NaN, Null, NaT, None, Missing, etc. en columnas numéricas se convierten en un float
df_titanic.head() # Muestra las primeras 5 filas del DataFrame

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(df_titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [5]:
print(df_titanic.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [6]:
# Existen tres tipos de datos faltantes:
# 1. Datos faltantes completamente al azar (MCAR): Datos que faltan sin ningún patrón específico.
# 2. Datos faltantes al azar (MAR): Datos que faltan debido a alguna variable observada.
# 3. Datos faltantes no al azar (MNAR): Datos que faltan debido a la variable misma.

# Identificar valores faltantes
datos_perdidos = df_titanic.isnull().sum() # Cuenta los valores faltantes en cada columna
print("Valores faltantes por columna:\n", datos_perdidos)

Valores faltantes por columna:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [7]:
# Ver los datos perdidos en en las columnas específicas
col_datos_perd = ['Age', 'Fare', 'Cabin']
print(df_titanic[col_datos_perd].isnull().sum())

Age      177
Fare       0
Cabin    687
dtype: int64


In [8]:
print(len(df_titanic)) # Número total de filas en el DataFrame

891


In [9]:
print(df_titanic[col_datos_perd].isnull().sum() / len(df_titanic) * 100) # el porcentaje se puede calcular usando una regla de tres simple


Age      19.865320
Fare      0.000000
Cabin    77.104377
dtype: float64


In [10]:
print(df_titanic.nunique())  # Cuenta los valores únicos en cada columna
# Ejemplo si en Sex hay mas de 2 valores unicos entonces hay un error en los datos

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64


In [11]:
# Agrupar por clase cuantas personas tienen registrado la edad (Pclass y Age)
edad_por_clase = df_titanic.groupby('Pclass')['Age'].count()
print(edad_por_clase) # Cuenta los valores no nulos en 'Age' para cada 'Pclass'


Pclass
1    186
2    173
3    355
Name: Age, dtype: int64


In [32]:
# Calcular cuantas personas hay por clase
total_pers_clase = df_titanic['Pclass'].value_counts().sort_index()
print(total_pers_clase) # Cuenta los valores en 'Pclass'
# Value_counts() devuelve una Serie con el conteo de ocurrencias de cada valor único en la columna especificada.

Pclass
1    107
2     93
3    218
Name: count, dtype: int64


In [33]:
resta = total_pers_clase - edad_por_clase
print(resta) # Resta los conteos para encontrar cuántas personas no tienen registrada

Pclass
1     9
2     5
3    72
dtype: int64


In [49]:
# isnull me devuelve un DataFrame de booleanos / serie de booleanos
sin_edad = df_titanic[df_titanic['Age'].isnull()]  # Filtra las filas donde 'Age' es nulo
print(sin_edad['Pclass'].value_counts().sort_index())


Pclass
1     9
2     5
3    72
Name: count, dtype: int64


In [50]:
print(sin_edad['Pclass'].value_counts().sort_index() / len(sin_edad) * 100) # Porcentaje de personas sin edad por clase

Pclass
1    10.465116
2     5.813953
3    83.720930
Name: count, dtype: float64


In [56]:
# Usemos estadistica central como el promedio, la mediana, y la moda para validar con que valor rellenar la columna Age
promedio_edad = df_titanic['Age'].mean()
print(promedio_edad)
mediana_edad = df_titanic['Age'].median() # la mediana me da el valor central de una distribución
print(mediana_edad)
moda_edad = df_titanic['Age'].mode()[0]
print(moda_edad)# La moda puede devolver múltiples valores, tomamos el primero
print(df_titanic['Age'].min(), df_titanic['Age'].max())

30.272590361445783
27.0
21.0
0.17 76.0


In [12]:
print(df_titanic.groupby('Sex')['Survived'].value_counts())

Sex     Survived
female  1           233
        0            81
male    0           468
        1           109
Name: count, dtype: int64


In [14]:
print(df_titanic['Sex'].value_counts())

Sex
male      577
female    314
Name: count, dtype: int64


In [20]:
resultado_sobrevivencia_sexo = df_titanic.groupby('Sex')['Survived'].agg(['mean', 'count', 'sum'])
resultado_sobrevivencia_sexo['mean'] *= 100  # Convertir a porcentaje el promedio de supervivencia

print(resultado_sobrevivencia_sexo)

             mean  count  sum
Sex                          
female  74.203822    314  233
male    18.890815    577  109


In [ ]:
# la columana edad es una variable cuantitativa continua
# Discretas y continuas
# Discretas: toman valores específicos y separados (ejemplo: número de hermanos, número de hijos)
# Continuas: pueden tomar cualquier valor dentro de un rango (ejemplo: altura, peso, temperatura)


In [23]:
print(df_titanic.nunique()) # Cuenta los valores únicos en cada columna

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64


In [30]:
# Crear una columna por rangos de edad
df_titanic['Grupo Age'] = pd.cut(df_titanic['Age'], bins=[0,12,18,30,60,100], labels = ['Niño', 'Adolescente', 'Joven Adulto', 'Adulto', 'Anciano'])
print(df_titanic['Grupo Age'].value_counts( dropna = False))

Grupo Age
Adulto          283
Joven Adulto    270
NaN             177
Adolescente      70
Niño             69
Anciano          22
Name: count, dtype: int64


In [31]:
print(df_titanic['Grupo Age'].value_counts())

Grupo Age
Adulto          283
Joven Adulto    270
Adolescente      70
Niño             69
Anciano          22
Name: count, dtype: int64
